In [1]:
import time
import ee
import geemap

import ee_imagery_downloader.utils as utils
from ee_imagery_downloader.eeImageryInterface import eeImageryInterface

%reload_ext autoreload
%autoreload 2

## Browse Earth Engine Imagery

This notebook demonstrates an interactive environment in which to explore & visualize imagery via `eeImageryInterface`.


### Configuration

In [2]:
roi = 'example_sanikiluaq'  # refer to image_filtering_configs

vis_start_date = "2020-03-02"
vis_end_date = "2020-03-05"

# Following the collection_filters, we want to visualize EW HH-HV S1 imagery
# (stored in a nested dict by eeImageryInterface) and un-grouped S2 imagery.
vis_imagery_query = {
    'S1': {
        'nested_keys': ['EW', 'HH-HV'],
        'vis_params': {
            'min':-40,
            'max':-5,
            'gamma':0.5,
            'bands': ['HH']
        }
    },
    'S2': {
        'nested_keys': [],
        'vis_params': {
            'min': 0,
            'max': 10000,
            'gamma':0.77,
            'bands': ['B4', 'B3', 'B2']  # Visible
            # 'bands': ['B8', 'B4', 'B2']  # NIR
        }
    }
}

In [3]:
# Load imagery
interface = eeImageryInterface(roi)
print(interface)

eeImageryInterface for example_sanikiluaq;
> Imagery loaded for seasons: 2020, 2022
> Structure of self.imagery (keyed by season):
{'S1': {'EW': {'HH-HV': <ee.imagecollection.ImageCollection object at 0x0000024533ED0C70>},
        'IW': {'VV': <ee.imagecollection.ImageCollection object at 0x0000024533ED0FD0>}},
 'S2': <ee.imagecollection.ImageCollection object at 0x0000024533FA48E0>}


In [4]:
def collToList(item, accumulator):
    return ee.List(accumulator).add(item)

def addCollectionToMap(coll_name, collection, Map, vis_params):
    collection_list = ee.List(collection.iterate(collToList, ee.List([])))
    for i in range(collection_list.length().getInfo()):
        image = ee.Image(collection_list.get(i))
        datetime = image.date().format('MMM dd HH:mm').getInfo()
        Map.addLayer(image, vis_params, f"{coll_name}_{datetime}")

# Create interactive map
roi_center_coords = utils.latlnFromPolygon(interface.ee_roi)
Map = geemap.Map(center=roi_center_coords, zoom=8)

# Retrieve & show imagery
filter_date_start = ee.Date(vis_start_date)
filter_date_end = ee.Date(vis_end_date)

for coll, cfg in vis_imagery_query.items():
    get_s = time.time()
    imagery = interface.getImagery(
        coll,
        nested_keys=cfg['nested_keys'],
        date_query = {
            'start': vis_start_date,
            'end': vis_end_date
        },
        mosaic_window='hour'  # Note that this can significantly slow down computation!
    )
    vis_params = cfg['vis_params']
    addCollectionToMap(coll, imagery, Map, vis_params)


Mosaicking S1 by hour/EW/HH-HV
Mosaicking S2 by hour


In [5]:
# Show the map
Map

Map(center=[56.2535289621146, -79.35250000000006], controls=(WidgetControl(options=['position', 'transparent_b…